### Keras Tuner Neural Network

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import tensorflow as tf
from sklearn.model_selection import train_test_split
from pathlib import Path

In [2]:
# Disable warnings
import warnings
warnings.filterwarnings('ignore')

In [12]:
def preprocessFile(filename = Path('../Data/AdultCensusIncome.csv')):
    
    print(f'Preprocessing file {filename}')
    dtype_mapper = {'age': 'int64',
                    'workclass': 'string',
                    'fnlwgt': 'int64',
                    'education': 'string',
                    'education.num': 'int64',
                    'marital.status': 'string',
                    'occupation': 'string',
                    'relationship': 'string',
                    'race': 'string',
                    'sex': 'string',
                    'capital.gain': 'int64',
                    'capital.loss': 'int64',
                    'hours.per.week': 'int64',
                    'native.country': 'string',
                    'income': 'string'}
    df = pd.read_csv(filename, dtype=dtype_mapper, na_values='?')    
    df.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'income']
    df.income = df.income.apply(lambda x: '0' if x == '<=50K' else '1')
    df.income = df.income.astype('int64')
    df = df.drop(['fnlwgt',
                  'education', 
                #   'capital_gain', 
                #   'capital_loss',
                  'native_country'], axis=1)
    df = df.dropna()
    return df

census_df = preprocessFile()
census_df.sample(10)

Preprocessing file ..\Data\AdultCensusIncome.csv


,age,workclass,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,income
27825,48,Self-emp-inc,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,40,1
4362,33,Private,13,Never-married,Exec-managerial,Other-relative,Asian-Pac-Islander,Female,0,0,40,0
30163,62,Private,6,Married-civ-spouse,Craft-repair,Husband,Black,Male,0,0,40,0
5692,37,Private,10,Divorced,Adm-clerical,Unmarried,Black,Female,0,0,35,0
13983,20,Private,9,Never-married,Adm-clerical,Not-in-family,White,Female,0,0,40,0
15813,28,Private,6,Never-married,Handlers-cleaners,Other-relative,White,Male,0,0,40,0
19301,18,Private,9,Never-married,Other-service,Own-child,White,Male,0,0,35,0
15676,21,Private,9,Never-married,Craft-repair,Not-in-family,White,Male,0,0,40,0
27901,39,Private,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,0
4855,22,Private,10,Never-married,Sales,Own-child,White,Female,0,0,15,0


In [13]:
y = census_df.income

X = census_df.copy()
X.drop('income', axis=1, inplace=True)

X = pd.get_dummies(X, dtype='int64')
X.head()

# Use sklearn to split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create scaler instance
X_scaler = skl.preprocessing.StandardScaler()

# Fit the scaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=30,
        step=5), activation=activation, input_dim=2))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=30,
            step=5),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [15]:
# Import the kerastuner library
import keras_tuner as kt
# Most popular tuner
tuner = kt.Hyperband(create_model, objective='val_accuracy', max_epochs=20, hyperband_iterations=2)


Reloading Tuner from .\untitled_project\tuner0.json


In [10]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled, y_train, epochs=20, validation_data = (X_test_scaled, y_test))

Trial 2 Complete [00h 00m 00s]

Best val_accuracy So Far: None
Total elapsed time: 00h 00m 01s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
tanh              |tanh              |activation
1                 |6                 |first_units
2                 |3                 |num_layers
1                 |26                |units_0
26                |1                 |units_1
6                 |1                 |units_2
21                |None              |units_3
21                |None              |units_4
3                 |3                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/3


Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\envs\dev\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\ProgramData\anaconda3\envs\dev\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "c:\ProgramData\anaconda3\envs\dev\lib\site-packages\keras_tuner\src\tuners\hyperband.py", line 427, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
  File "c:\ProgramData\anaconda3\envs\dev\lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "c:\ProgramData\anaconda3\envs\dev\lib\site-packages\keras_tuner\src\engine\tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File 

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\envs\dev\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\ProgramData\anaconda3\envs\dev\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "c:\ProgramData\anaconda3\envs\dev\lib\site-packages\keras_tuner\src\tuners\hyperband.py", line 427, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
  File "c:\ProgramData\anaconda3\envs\dev\lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "c:\ProgramData\anaconda3\envs\dev\lib\site-packages\keras_tuner\src\engine\tuner.py", line 233, in _build_and_fit_model

In [16]:
# Get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
  print(param.values)

{'activation': 'relu', 'first_units': 16, 'num_layers': 5, 'units_0': 26, 'units_1': 26, 'units_2': 11, 'tuner/epochs': 3, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0, 'units_3': 1, 'units_4': 1}
{'activation': 'tanh', 'first_units': 1, 'num_layers': 2, 'units_0': 1, 'units_1': 26, 'units_2': 6, 'units_3': 21, 'units_4': 21, 'tuner/epochs': 3, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}
{'activation': 'tanh', 'first_units': 6, 'num_layers': 3, 'units_0': 26, 'tuner/epochs': 3, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0, 'units_1': 1, 'units_2': 1}
